## FINAL PROJECT

### **Цель**
##### Подготовить основу рекомендательной системы.
### **Задачи**
##### 1. Познакомиться с датасетом, подготовить и проанализировать данные с помощью SQL.
##### 2. Обработать данные средствами Python.
##### 3. Составить итоговую таблицу с рекомендациями, снабдив её необходимыми комментариями, и представить отчёт продакт-менеджеру.

    Изучим данные, полученные из 2-х таблиц:
    carts с данными о пользовательских корзинах (дате создания, статусе, id пользователя-владельца и т. д.);
    cart items с данными о курсах, которые пользователи добавили в корзину.

    Напишем SQL-запрос, чтобы выгрузить данные по всем продажам курсов в разрезе пользователей.

With new as
(Select 
user_id,
count(distinct resource_id)

from final.cart_items i
left join final.carts c on c.id = i.cart_id
where state = 'successful' and resource_type = 'Course'
group by user_id
having count(distinct resource_id) > 1)

select 
new.user_id,
i.resource_id
from new 
left join final.carts c on new.user_id = c.user_id
join final.cart_items i on i.cart_id = c.id
WHERE state = 'successful' AND resource_type = 'Course'
group by 1, 2
order by 1, 2

    Подготовили файл с данными по продажам курсов в разрезе пользователей (купивших более одного курса). Выгрузим получившуюся таблицу в csv-файл для дальнейшей работы.

In [1]:
import pandas as pd # Импортируем библиотеки
import numpy as np

data = pd.read_csv('query_result.csv') # Загрузим данные из подготовленного файла
df = data.copy()
df.head() # Отобразим получившуюся таблицу

,user_id,resource_id
0,51,516
1,51,1099
2,6117,356
3,6117,357
4,6117,1125


In [2]:
df.info() # Посмотрим основную информацию о нашей таблице

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34074 entries, 0 to 34073
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   user_id      34074 non-null  int64
 1   resource_id  34074 non-null  int64
dtypes: int64(2)
memory usage: 532.5 KB


In [3]:
# Создадим сводную таблицу, в которой отобразим список купленных курсов в разрезе покупателей.
resourse_df = df.groupby('user_id')['resource_id'].agg(list).reset_index()
resourse_df.head()

,user_id,resource_id
0,51,"[516, 1099]"
1,6117,"[356, 357, 1125]"
2,10275,"[553, 1147]"
3,10457,"[361, 1138]"
4,17166,"[356, 357]"


In [4]:
# По заданию определим сколько различных пар курсов встречаются вместе в покупках клиентов.
from itertools import combinations # Выгрузим библиотеку
cource = [] # Создадим пустой список для хранения уникальных значений всех возможных пар курсов
for i in resourse_df['resource_id']:
    for pair_id in combinations(i,2):
        cource.append(pair_id)

from collections import Counter
cources = Counter(cource).most_common()
print("Количество различных пар курсов -", len(cources))        

# Для дальнейшего анализа преобразуем полученный список в словарь:
popular = dict(cources)

Количество различных пар курсов - 3989


In [15]:
print('ID курсов в самой популярной паре:', max(popular, key=popular.get))

ID курсов в самой популярной паре: (551, 566)


In [6]:
# Составим список продаваемых курсов:
uniq_course = []
for i in df['resource_id']:
    if not(i in uniq_course):
        uniq_course.append(i)       

In [7]:
# По частоте покупок определим малопопулярные курсы, для этого по 75-й квантили установим минимальную границу.
pair_purchase = pd.DataFrame(cources, columns=['pair', 'purchase'])
pair_purchase_quantile = pair_purchase['purchase'].quantile(0.75)
pair_purchase_quantile

9.0

По полученному результату приходим к тому, что курсы с частотой меньше 9 для заполнения учитывать не будем.

In [8]:
# Создадим функцию, через которую будем анализировать созданный словарь с парами курсов и частотой покупки этих пар
def recommend(pair):
    course_list = [] # Содается пустой список
    for i in popular.keys(): # Через цикл перебираются ключи словаря list_paires
# В случае нахождения значения(пары курсов - частоты продажи), оно добавляется в список в виде: ключ - пара курсов, значение - частота покупки
        if i[0] == pair and popular[i] >= pair_purchase_quantile:
                course_list.append((i, popular[i]))
        if i[1] == pair and popular[i] >= pair_purchase_quantile:
                course_list.append((i[::-1], popular[i]))

    sort_list = sorted(course_list, key=lambda x: x[1], reverse=True) # Отсортируем частоту продажи по убыванию
    return sort_list[:2] # Оставим только два курса

# Создаем дата фрейм, в который будут добавляться данные
cource_df = pd.DataFrame(columns=['Course', 'first_recommendation', 'second_recommendation'])

# Через цикл происходит анализ списка, содержащего перечень уникальных курсов и в дата фрейм добавляется значение анализируемого курса, курса "первой рекомендации" 
# и курса "второй рекомендации"

for i in uniq_course:
    if len(recommend(i)) == 2:
        cource_df.loc[i] = [i, recommend(i)[0][0][1], recommend(i)[1][0][1]]
    elif len(recommend(i)) == 1:
        cource_df.loc[i] = [i, recommend(i)[0][0][1], np.nan]
    else:
        cource_df.loc[i] = [i, np.nan, np.nan]
cource_df = cource_df.sort_values(['Course']).reset_index(drop=True)

In [9]:
cource_df.head() # Выведем получившуюся таблицу.

,Course,first_recommendation,second_recommendation
0,356.0,571.0,357.0
1,357.0,571.0,356.0
2,358.0,570.0,752.0
3,359.0,570.0,358.0
4,360.0,745.0,516.0


Оставшиеся пропуски в таблице с рекомендациями заполним более новыми курсами с высокой частотой продаж. Новыми курсами будем считать, курсы, у которых номера начинаются с 1100.

In [10]:
# Создадим сводную таблицу по количеству продаж в разрезе курсов:
courses_df = df.groupby('resource_id')['user_id'].nunique().reset_index().sort_values('user_id', ascending=False)
courses_df.columns = ['resource_id', 'purchase'] # Переименуем столбцы в нашей таблице
courses_df.sort_values('purchase') # Отсортируем по убыванию
courses_df.head() # Отобразим получившуюся таблицу

,resource_id,purchase
30,551,2935
36,566,2342
24,515,1311
13,489,1125
14,490,1084


In [11]:
# Выберем курсы, которые начинаются с 1100 и определим из них самые покупаемые:
top = courses_df[courses_df['resource_id'] >= 1100]
top_1 = top['resource_id'].iloc[0]
top_2 = top['resource_id'].iloc[1]

In [12]:
# Заменим пропуски на курсы, которые мы определиили "топовыми"
cource_df['first_recommendation'] = cource_df['first_recommendation'].fillna(top_1)
cource_df['second_recommendation'] = cource_df['second_recommendation'].fillna(top_2)

In [13]:
# Отобразим нашу итоговую таблицу с рекомендациями к каждому курсу:
display(cource_df)

,Course,first_recommendation,second_recommendation
0,356.0,571.0,357.0
1,357.0,571.0,356.0
2,358.0,570.0,752.0
3,359.0,570.0,358.0
4,360.0,745.0,516.0
...,...,...,...
121,1188.0,1141.0,513.0
122,1198.0,1103.0,1100.0
123,1199.0,1103.0,1100.0
124,1200.0,1103.0,1100.0


In [14]:
cource_df.info() # Посмотрим информацию по итоговой таблице на наличие пропусков.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Course                 126 non-null    float64
 1   first_recommendation   126 non-null    float64
 2   second_recommendation  126 non-null    float64
dtypes: float64(3)
memory usage: 3.1 KB


 ### В результате проделанной работы получили рекомендательную таблицу, в которой каждому продаваемому курсу рекомендуем дополнительные курсы. Первым рекомендуемым курсом, будет курс, который продавался чаще всего в паре с анализируемым. Вторым рекомендуемым курсом будет второй по продаваемости. Где продаж в паре было не много, чтобы исключить случайные покупки, решили рекомендовать новые курсы с наибольшим количество продаж.